In [6]:
import pandas as pd
import spacy
from spacy.pipeline import EntityRuler
import tqdm
import glob
import os

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Add your custom EntityRuler
ruler = nlp.add_pipe("entity_ruler", before="ner")

In [7]:
#Init Variables for csv names
year="2024"
# month="01"
cwd=os.getcwd()
month=os.path.basename(cwd)
print(f"{month}")

special_identifier='_com_air_hea_tra_ban_us_er_yt_red_net' #for csv output

services = [
    "Comcast", "Airline", "Healthcare", "Trains", "Banks", "United States",
    "ER", "Youtube", "Reddit", "Netflix"
    # "Xfinity", "Amtrak", "Septa", "Spirit", "American"
]

complaint_patterns = [{"label": "SERVICE", "pattern": service} for service in services]


06


In [3]:
# Your patterns
# complaint_patterns = [
#     {"label": "SERVICE", "pattern": "Comcast"},
#     {"label": "SERVICE", "pattern": "Airline"},
#     {"label": "SERVICE", "pattern": "Healthcare"},
#     {"label": "SERVICE", "pattern": "Trains"},
#     {"label": "SERVICE", "pattern": "Banks"},
#     {"label": "SERVICE", "pattern": "United States"},
#     {"label": "SERVICE", "pattern": "ER"},
#     {"label": "SERVICE", "pattern": "Youtube"},
#     {"label": "SERVICE", "pattern": "Reddit"},
#     {"label": "SERVICE", "pattern": "Lunar Coin"}
# ]

ruler.add_patterns(complaint_patterns)

# Load data
df = pd.read_csv(f"../../FILTERED_SUBMISSIONS_bad/RS_{year}-{month}.csv", names=["id", "date", "title", "author", "url", "content", "post_id", "timestamp", "subreddit"])

df.head()

,id,date,title,author,url,content,post_id,timestamp,subreddit
0,1,2023-12-31,I need to talk,u/Entire_Wrongdoer_780,https://www.reddit.com/r/mentalhealth/comments...,I need to free myself from this burden that ea...,18vkgs2,1.704067e+09,t5_2qirg
1,1,2023-12-31,In the planning stages now,u/Rando-name2023,https://www.reddit.com/r/SuicideWatch/comments...,I don't know if things have ever been this bad...,18vkgsw,1.704067e+09,t5_2qpzs
2,1,2023-12-31,good girls <3,u/s-exhibitionism,https://www.reddit.com/r/EdgingTalk/comments/1...,"\nyou’re all doing soo well! keep going, keep ...",18vkgv7,1.704067e+09,t5_3a6db
3,1,2023-12-31,"Farrier here, need financial advice",u/PigArmy,https://www.reddit.com/r/Farriers/comments/18v...,"Hey Guys/Gals, I’m a farrier in Northeaster Oh...",18vkgvh,1.704067e+09,t5_2zvj3
4,1,2023-12-31,Sobre A Odisséia de Homero,u/JacquesTimmermans,https://www.reddit.com/r/u_JacquesTimmermans/c...,https://i.redd.it/2mmortupyp9c1.jpeg,18vkgvt,1.704067e+09,t5_91lb9d


In [2]:
# services = [
#     "Comcast", "Airline", "Healthcare", "Trains", "Banks", "United States",
#     "ER", "Youtube", "Reddit", "Lunar Coin"
# ]

# Build regex pattern for fast searching (case insensitive)
pattern = r'|'.join(services)

# Filter rows where content mentions any service
filtered_df = df[df['content' ].str.contains(pattern, case=False, na=False) |
                df['subreddit'].str.contains(pattern, case=False, na=False) ]

print(f"Original size: {len(df)}")
print(f"Filtered size: {len(filtered_df)}")

Original size: 530792
Filtered size: 468409


In [3]:
neg_words = ["bad", "terrible", "lazy", "worst", "awful", "scam", "horrible", "broken", "slow"]

pattern_neg = r'|'.join(neg_words)

complaint_df = filtered_df[
    filtered_df['content'].str.contains(pattern_neg, case=False, na=False)
]

complaint_df.head()


,id,date,title,author,url,content,post_id,timestamp,subreddit
0,1,2024-11-30,Symptoms??,u/astrofoxical,https://www.reddit.com/r/TryingForABaby/commen...,I'm a mom of 1 and currently TTC but I just n...,1h3q1vy,1.733011e+09,t5_2sil5
2,1,2024-11-30,Raincheck,u/MediumBarnacle1438,https://www.reddit.com/r/Unsent_Unread_Unheard...,That was ALWAYS you \nWhen I saw on unsent thi...,1h3q21s,1.733011e+09,t5_8ltvmn
3,1,2024-11-30,"I have CPTSD and my new ""trauma"" therapist tri...",u/Far-Illustrator-4008,https://www.reddit.com/r/TalkTherapy/comments/...,I have CPTSD and I’m just a few sessions in (a...,1h3q23r,1.733011e+09,t5_39dpu
4,1,2024-11-30,Will I be hindered from starting the manga fro...,u/Espurrizumi,https://www.reddit.com/r/StardustCrusaders/com...,I am a huge fan of jojos and have been for yea...,1h3q27e,1.733011e+09,t5_2tny5
5,1,2024-11-30,I don't know how to go forward.,u/DisappointedInMyseIf,https://www.reddit.com/r/BreakUps/comments/1h3...,"This is going to be long, I haven't been able ...",1h3q298,1.733011e+09,t5_2ra79


In [4]:
complaint_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 435182 entries, 0 to 530791
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         435182 non-null  int64  
 1   date       435182 non-null  object 
 2   title      435181 non-null  object 
 3   author     435182 non-null  object 
 4   url        435182 non-null  object 
 5   content    435182 non-null  object 
 6   post_id    435182 non-null  object 
 7   timestamp  435182 non-null  float64
 8   subreddit  435182 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 33.2+ MB


In [5]:
from tqdm.notebook import tqdm

In [ ]:
batch_size = 5000
max_length = 1000  # Skip posts longer than this
skipped_posts = []

for start in tqdm(range(0, len(df), batch_size)):
    end = start + batch_size
    batch = df.iloc[start:end].copy()

    # Only keep rows with short enough content
    batch = batch[batch['content'].str.len() < max_length]

    texts = batch['content'].tolist()
    ner_results = []

    for doc, row in zip(nlp.pipe(texts, batch_size=50), batch.itertuples()):
        try:
            ents = [(ent.text, ent.label_) for ent in doc.ents]
        except Exception as e:
            print(f"Skipping post {row.post_id} due to error: {e}")
            skipped_posts.append(row.post_id)
            ents = []

        ner_results.append(ents)

    batch['entities'] = ner_results

    batch.to_csv(f'ner_results_batch_{year}.{month}_{start}.csv', index=False)

# Save skipped post IDs
pd.Series(skipped_posts).to_csv('skipped_posts.csv', index=False)

print("NER complete.")


  0%|          | 0/107 [00:00<?, ?it/s]

In [8]:
import glob

# Load all batch files
files = glob.glob("ner_results_batch_*.csv")

dfs = [pd.read_csv(f) for f in files]

final_ner_df = pd.concat(dfs, ignore_index=True)
final_ner_df.to_csv(f'ner_results_append_sum_{year}.{month}{special_identifier}.csv', index=False)


In [9]:
final_ner_df.head()

,id,date,title,author,url,content,post_id,timestamp,subreddit,entities
0,22,2024-05-31,how to rehydrate/ avoid a dehydration headache,u/New-Commercial7532,https://www.reddit.com/r/HydroHomies/comments/...,"I feel a headache coming on, and i have to go ...",1d59xy1,1.717200e+09,t5_10288s,"[('5ish hours', 'TIME'), ('3', 'CARDINAL')]"
1,1,2024-05-31,This guy I talk to texts me to wish my son a H...,u/youwotm8123456781,https://www.reddit.com/r/casualdating/comments...,"I'm talking to this guy for a few weeks, but I...",1d59y5w,1.717200e+09,t5_359pr,"[('a few weeks', 'DATE'), ('Crip', 'ORG'), ('B..."
2,250,2024-05-31,Some pretty ladies I've come across while working,u/FreeCrayons,https://www.reddit.com/r/spiders/comments/1d59...,https://www.reddit.com/gallery/1d59ye2,1d59ye2,1.717200e+09,t5_2r1sf,[]
3,1,2024-05-31,I think I fucked up!,u/nativesage19,https://www.reddit.com/r/CreatorsAdvice/commen...,https://www.reddit.com/gallery/1d59yir,1d59yir,1.717200e+09,t5_4xgztn,[]
4,1,2024-05-31,separation anxiety,u/Prestigious-Tie-8906,https://www.reddit.com/r/Anxiety/comments/1d59...,"I’m 23, my dad was never there for me as a kid...",1d59ykm,1.717200e+09,t5_2qmij,"[('23', 'DATE'), ('99%', 'PERCENT')]"


In [10]:
from collections import Counter
import ast

entity_counter = Counter()

for entities in final_ner_df['entities']:
    ents = ast.literal_eval(entities)
    entity_counter.update([e[0] for e in ents])

print(entity_counter.most_common(20))


[('first', 21825), ('one', 14807), ('2', 13300), ('today', 10193), ('3', 9263), ('two', 8959), ('1', 8610), ('4', 5308), ('5', 5249), ('second', 4887), ('yesterday', 3559), ('10', 3030), ('the day', 2925), ('6', 2692), ('years', 2589), ('English', 2588), ('One', 2437), ('Idk', 2382), ('tomorrow', 2253), ('20', 2215)]


In [11]:
subreddit_counter = final_ner_df['subreddit'].value_counts()

print(subreddit_counter.head(20))


subreddit
t5_2vqun     1715
t5_2qjdm     1584
t5_33rc6     1555
t5_122hf1    1475
t5_2rjli     1303
t5_2w844     1204
t5_3pvrd     1184
t5_2xtuc     1095
t5_2qpzs      973
t5_2vfcm      949
t5_7f6cy1     943
t5_2qmij      912
t5_294odh     906
t5_2ranw      756
t5_2tkvu      751
t5_75flg6     742
t5_2qqqf      719
t5_2sxk3      664
t5_2ra79      663
t5_2r749      641
Name: count, dtype: int64


In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


In [ ]:
def get_sentiment(text):
    if isinstance(text, str):
        return analyzer.polarity_scores(text)['compound']
    return 0.0  # Neutral if empty or NaN

final_ner_df['sentiment'] = final_ner_df['content'].apply(get_sentiment)


In [ ]:
final_ner_df.to_csv(f'ner_results_append_sum_{year}_{month}_sentiment{special_identifier}.csv', index=False)

In [ ]:
# Extract SERVICE entities only
service_sentiments = []

for row in final_ner_df.itertuples():
    ents = ast.literal_eval(row.entities)
    services = [e[0] for e in ents if e[1] == 'SERVICE']
    for service in services:
        service_sentiments.append((service, row.sentiment))

service_df = pd.DataFrame(service_sentiments, columns=['service', 'sentiment'])

service_avg_sentiment = service_df.groupby('service')['sentiment'].mean().sort_values()

print(service_avg_sentiment)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.barplot(x=service_avg_sentiment.values, y=service_avg_sentiment.index)
plt.title("Average Sentiment by Service (VADER)")
plt.xlabel("Sentiment Score")
plt.ylabel("Service")
plt.tight_layout()
plt.show()


In [ ]:
final_ner_df['date'] = pd.to_datetime(final_ner_df['date'])  # or 'timestamp' column if you used that

In [ ]:
service_sentiments = []

for row in final_ner_df.itertuples():
    ents = ast.literal_eval(row.entities)
    services = [e[0] for e in ents if e[1] == 'SERVICE']
    for service in services:
        service_sentiments.append((row.date, service, row.sentiment))

service_time_df = pd.DataFrame(service_sentiments, columns=['date', 'service', 'sentiment'])


In [ ]:
# Resample weekly for smoother lines
service_time_df = service_time_df.set_index('date')

avg_sentiment_time = service_time_df.groupby(['service']).resample('7D').mean().reset_index()


In [ ]:
plt.figure(figsize=(12, 7))

sns.lineplot(
    data=avg_sentiment_time,
    x='date',
    y='sentiment',
    hue='service'
)

plt.title("Average VADER Sentiment Over Time by Service")
plt.xlabel("Date")
plt.ylabel("Sentiment Score")
plt.legend(title='Service', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
post_freq = final_ner_df.set_index('date').resample('1D').size().reset_index(name='post_count')

plt.figure(figsize=(12, 6))

sns.lineplot(data=post_freq, x='date', y='post_count')

plt.title("Number of Reddit Posts Over Time")
plt.xlabel("Date")
plt.ylabel("Post Count")
plt.tight_layout()
plt.show()
